In [1]:
%load_ext autoreload
%autoreload 2
import logging
import pathlib
from typing import Type
from datetime import  datetime
import os
#from croston import croston
# from timeseries.models.deepar import DeepAR

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import click
import numpy as np
import pandas as pd
from gpflow import default_float
from gpflow.kernels import Kernel
from gpflow.likelihoods import Likelihood
from timeseries.errors import compute_errors
from timeseries.likelihoods import NegativeBinomial
from timeseries.models.gaussian_process import GaussianProcess
from timeseries.models.kernels import Kernel_Comb3#, Kernel_Comb1, Kernel_Comb2, MaternTimeseriesKernel
from timeseries.timeseries import Timeseries
def get_kernel(kernel) -> Type[Kernel]:
    if kernel == "Kernel_Comb3":
        return Kernel_Comb3
    # if kernel == "Kernel_Comb1":
    #     return Kernel_Comb1
    # if kernel == "Kernel_Comb2":
    #     return Kernel_Comb2
    # if kernel == "Matern":
        # return MaternTimeseriesKernel
    raise ValueError("wrong kernel specified")


def get_likelihood(likelihood) -> Type[Likelihood]:
    if likelihood == "NB":
        return NegativeBinomial
    raise ValueError("wrong kernel specified")

### GP method

In [2]:
def GP_fit_forecast(x_train, x_test, y_train, y_test, prediction_length,
                    kernel='Kernel_Comb3', likelihood='NB', num_restarts=3, use_priors=True):
    
    m = GaussianProcess(kernel=get_kernel(kernel),
                        likelihood=get_likelihood(likelihood),use_priors=use_priors)
    
    elboopt = -np.inf
    for i in range(num_restarts):
        m.fit(y_train, x_train,
                  prediction_length=h, iterations=5000, n_batch=256)
        elbo = np.array(m._model.elbo((x_train,y_train)))
        if elboopt<elbo:
            elboopt=elbo
    
    return m.forecast(y_test, x_test)

### Prepare and create a subset of the data

In [3]:
_DP = '../../../data/'
MP = '../../../models/'
ste = pd.read_pickle(f'{_DP}processed/sales_train_evaluation.pkl')
x = np.load(f'{_DP}processed/time_x.npy')

In [4]:
sampled_ids = np.load(f'{_DP}interim/sampled_ids.npy').tolist()

# -------------------------------------------

In [5]:
sampled_ste = \
    ste \
    .reset_index('id') \
    .reset_index() \
    .set_index('id') \
    .loc[sampled_ids] \
    .reset_index()

In [6]:
sampled_y = sampled_ste.set_index('id') \
    .iloc[:, 1:] \
    .astype('float64') \
    .to_numpy() \
    .transpose()

In [7]:
all_y = sampled_y

# GP

In [8]:
m_name = 'GP_305'
DP = MP
model_path = f'{DP}{m_name}/'
os.makedirs(model_path, exist_ok=True)

Run the following cell only once, then comment out if you continue the code another time:

In [9]:
%%time
prediction_length = 28
################################ NOTE: ########################################
# n_ys is hard coded here AND in gaussian_process.py. If you change #
# its numerical value here, make sure to change it in gaussian_process.py too #
###############################################################################
n_ys = 500

remaining_IDs = np.arange(all_y.shape[1])

pred_shape = (prediction_length, all_y.shape[1])
pmf_pred_shape = (n_ys, prediction_length, all_y.shape[1])
means = -1 * np.ones(pred_shape)
variances = -1 * np.ones(pred_shape)
pmfs = -1 * np.ones(pmf_pred_shape)

np.save(f'{model_path}remaining_IDs.npy', remaining_IDs)
np.save(f'{model_path}means.npy', means)
np.save(f'{model_path}variances.npy', variances)
np.save(f'{model_path}pmfs.npy', pmfs)

Wall time: 355 ms


In [10]:
means = np.load(f'{model_path}means.npy')
variances = np.load(f'{model_path}variances.npy')
remaining_IDs = np.load(f'{model_path}remaining_IDs.npy')
pmfs = np.load(f'{model_path}pmfs.npy')

In [11]:
import time

To start forecasting, run the following cell:

Once you want to turn off the script and save the preliminary results before all forecasts are completed, interrup the kernel (`I`+`I` keyboard shortcut), and run the two cells following this cell. To continue forecasting, simply re run this notebook, skipping the cell which initialised means, variances, pmfs, remaining_IDs.

In [13]:
%%time
print_interval = 5
save_interval = 50

prediction_length = 28
h = prediction_length
# commented these out since we are not comparing to SkewGP anymore, so no need to include only the last 250 time steps
# ncut = 1913
# ncut = 250
# ncutph = ncut + h
ncutph = all_y.shape[0]
x_train, x_test = x[-ncutph:-h], x[-h:]
start = time.perf_counter()
print('#'*40)
print(f'RUNNING THE {m_name} METHOD:')
print('#'*40)
for i, ID in enumerate(remaining_IDs):
    ind_nz = ncutph - np.where(all_y[-ncutph:,ID]>0)[0][0]
    ncutph0 = np.minimum(ncutph,ind_nz)
    y_train = all_y[-ncutph0:-h, ID][:, np.newaxis]
    y_test = all_y[-h:, ID][:, np.newaxis]
    x_train_temp = x[-ncutph0:-h][:]

    y_hat, y_hat_var, y_pmf = GP_fit_forecast(x_train_temp, x_test, y_train, y_test, h, num_restarts=3)#, kernel='Kernel_Comb1')

    
    means[:, ID][:, np.newaxis] = y_hat
    variances[:, ID][:, np.newaxis] = y_hat_var
    pmfs[:, :, ID][:, :, np.newaxis] = y_pmf

    last_idx = i
    if ID % print_interval == 0:
        print(f'Finished forecasting sales for all items up to (inclusive) item {ID}')
    
    if (i + 1) % save_interval == 0:
        np.save(f'{model_path}means.npy', means)
        np.save(f'{model_path}variances.npy', variances)
        np.save(f'{model_path}pmfs.npy', pmfs)
        temp_new_remaining_IDs = remaining_IDs[i+1:]
        np.save(f'{model_path}remaining_IDs.npy', temp_new_remaining_IDs)
        
        print('#' * 40)
        print(f'{save_interval} more forecasts completed. Saving means, variances, pmfs, and new_remaining_IDs.')
        print(f'{len(temp_new_remaining_IDs)} forecasts left.')
        temp_end = time.perf_counter()
        print(f'Script has been running for {np.round((temp_end - start)/60, 0)} minutes.')
        print('#' * 40)

########################################
RUNNING THE GP_305 METHOD:
########################################


C:\Users\sebas\AppData\Roaming\Python\Python39\site-packages\scipy\stats\_discrete_distns.py:307: RuntimeWarning: divide by zero encountered in _nbinom_pdf
  return _boost._nbinom_pdf(x, n, p)


Finished forecasting sales for all items up to (inclusive) item 0
Finished forecasting sales for all items up to (inclusive) item 5
Finished forecasting sales for all items up to (inclusive) item 10
Finished forecasting sales for all items up to (inclusive) item 15
Finished forecasting sales for all items up to (inclusive) item 20
Finished forecasting sales for all items up to (inclusive) item 25
Finished forecasting sales for all items up to (inclusive) item 30
Finished forecasting sales for all items up to (inclusive) item 35
Finished forecasting sales for all items up to (inclusive) item 40
Finished forecasting sales for all items up to (inclusive) item 45
########################################
50 more forecasts completed. Saving means, variances, pmfs, and new_remaining_IDs.
255 forecasts left.
Script has been running for 26.0 minutes.
########################################
Finished forecasting sales for all items up to (inclusive) item 50
Finished forecasting sales for all ite

In [14]:
end = time.perf_counter()

In [15]:
new_remaining_IDs = remaining_IDs[last_idx+1:]
total_time = end - start
tss_forecasted = len(remaining_IDs) - len(new_remaining_IDs)
time_per_ts = total_time / tss_forecasted

print(f'Started with {len(remaining_IDs)} time series to forecast,',
      f'now left with {len(new_remaining_IDs)} time series to forecast. In total, {305-len(new_remaining_IDs)}',
      f"time series' have been forecasted")
print()
print(f"Forecasting these {tss_forecasted} time series' took about {total_time:.0f} seconds",
     f', or {time_per_ts:.2f} seconds per time series.') 
print()
print(f'If this speed were to continue, it would',
     f'take {(time_per_ts * len(new_remaining_IDs)) / 3600 :.2f} hours to forecast the rest of the',
     f"time series'")

np.save(f'{model_path}remaining_IDs.npy', new_remaining_IDs)
np.save(f'{model_path}means.npy', means)
np.save(f'{model_path}variances.npy', variances)
np.save(f'{model_path}pmfs.npy', pmfs)

Started with 305 time series to forecast, now left with 0 time series to forecast. In total, 305 time series' have been forecasted

Forecasting these 305 time series' took about 9652 seconds , or 31.64 seconds per time series.

If this speed were to continue, it would take 0.00 hours to forecast the rest of the time series'


In [ ]:
# def pmf_to_cmf(pmfs):
#     cmfs = -1 * np.ones(pmfs.shape)
#     cmfs[0] = pmfs[0]
#     for y in range(1, pmfs.shape[0]):
#         cmfs[y] = cmfs[y-1] + pmfs[y]
#     return cmfs

# pmf_to_cmf(pmfs[:, :, 68])

array([[0.10228911, 0.10285551, 0.10345286, ..., 0.12237464, 0.12332276,
        0.12426703],
       [0.17304491, 0.17392735, 0.17485787, ..., 0.20389801, 0.2053304 ,
        0.20675317],
       [0.23099044, 0.23207474, 0.2332182 , ..., 0.26848702, 0.27020521,
        0.27190803],
       ...,
       [0.99177307, 0.99138946, 0.99100096, ..., 0.98365207, 0.983427  ,
        0.98320468],
       [0.99194477, 0.9915657 , 0.99118162, ..., 0.98388999, 0.98366568,
        0.98344407],
       [0.99211239, 0.9917378 , 0.9913581 , ..., 0.98412349, 0.98389995,
        0.98367905]])